## Consultas para el desarrollo de la API

En este notebook se crean las funciones para los endpoints que se consumirán en la API.

Tenemos 5 funciones para las consultas:

1) def **PlayTimeGenre**( genero : str )

2) def **UserForGenre**( genero : str )

3) def **UsersRecommend**( año : int )

4) def **UsersWorstDeveloper**( año : int )

5) def **sentiment_analysis**( empresa desarrolladora : str )

In [2]:
import pandas as pd

#### ENDPOINT 1

def **PlayTimeGenre**( genero : str ): Debe devolver año con mas horas jugadas para dicho género.

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [27]:
def PlayTimeGenre(genero: str):
    df = pd.read_csv('PlayTimeGenre.csv')
    
    # Agrupa por género y encuentra el índice (idxmax()) correspondiente fila con el  valor máximo en la columna 'hours_game' para cada grupo
    max_indices = df.groupby('genres')['Hours_played'].idxmax()

    # Seleccionar las filas correspondientes a los índices encontrados
    df_resultante = df.loc[max_indices]
    # Encuentra el año con la mayor cantidad de horas jugadas para un género específico
    year_max_Hours_played = df_resultante.loc[df_resultante['genres'] == genero, 'release_year'].iloc[0]
    
    # Construye el response_data
    rpta1 = {"Año de lanzamiento con más horas jugadas para Género '{}'"":" "{}".format(genero, year_max_Hours_played)}

    return rpta1

In [30]:
PlayTimeGenre('Sports')

{"Año de lanzamiento con más horas jugadas para Género 'Sports':2015"}

#### ENDPOINT 2

def **UserForGenre**( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [63]:
def UserForGenre( genero:str ):
    df2 = pd.read_csv('UserForGenre.csv',low_memory=False)

    df2_genre = df2[df2['genres']==genero]
    # Agrupar por 'genres', 'user_id' y 'release_year' y sumar 'Hours_played'
    df3_genre = df2_genre.groupby(['genres', 'user_id'], as_index=False)['Hours_played'].sum()
   
    # Encuentra el índice del usuario con el máximo número de Hours_played
    indice_max_hours = df3_genre['Hours_played'].idxmax()
    # Obtiene el user_id con mayor 'Hours_played' para el genero indicado
    user_id_max_hours = df3_genre.loc[indice_max_hours, 'user_id']

    df2_genre = df2_genre[df2_genre['user_id']==user_id_max_hours]
    # Filtra por 'user_id' igual al user_id con maximo numeor de horas jugadas y selecciona las columnas 'release_year' y 'Hours_played'
    df2_genre1 = df2_genre[df2_genre['user_id']==user_id_max_hours][['release_year', 'Hours_played']]
    
    # Actualiza nombre de columnas
    df2_genre1 = df2_genre1.rename(columns={'release_year': 'Año', 'Hours_played': 'Horas'})

    # Listamos como diccionario las 'Hours_played' por año
    dict_year_hours = df2_genre1.to_dict(orient='records') #  'records' significa que cada fila del DataFrame se convertirá en un diccionario

    # Crea el diccionario de retorno
    rpta2 = {
        "Usuario con más horas jugadas para Género {}".format(genero): user_id_max_hours,
        "Horas jugadas": dict_year_hours}

    return rpta2
    

In [65]:
UserForGenre('Action')

{'Usuario con más horas jugadas para Género Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1995, 'Horas': 3.62},
  {'Año': 1999, 'Horas': 0.73},
  {'Año': 2000, 'Horas': 1177.4},
  {'Año': 2001, 'Horas': 0.22},
  {'Año': 2002, 'Horas': 3.97},
  {'Año': 2003, 'Horas': 127.88},
  {'Año': 2004, 'Horas': 2123.52},
  {'Año': 2005, 'Horas': 355.65},
  {'Año': 2006, 'Horas': 14.93},
  {'Año': 2007, 'Horas': 1879.73},
  {'Año': 2008, 'Horas': 3.73},
  {'Año': 2009, 'Horas': 1805.43},
  {'Año': 2010, 'Horas': 1301.38},
  {'Año': 2011, 'Horas': 2581.6},
  {'Año': 2012, 'Horas': 6304.94},
  {'Año': 2013, 'Horas': 2007.68},
  {'Año': 2014, 'Horas': 2178.18},
  {'Año': 2015, 'Horas': 5208.52},
  {'Año': 2016, 'Horas': 492.93},
  {'Año': 2017, 'Horas': 722.12}]}

#### ENDPOINT 3

def **UsersRecommend**( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [129]:
def UsersRecommend(anio: int):
    df3 = pd.read_csv('UsersRecommend.csv')
    
    # Filtra por el año indicado
    df3_year = df3[df3['year']==anio]
    
    # selecciona las tres filas con los valores mayores en la columna 'number_user_id_recom'
    df3_year_top3 = df3_year.nlargest(3, 'number_user_id_recom')

    # Armamos la estuctura de respuesta
    rpta3 = [{"Puesto 1": df3_year_top3.iloc[0]['app_name']},{"Puesto 2": df3_year_top3.iloc[1]['app_name']},{"Puesto 3": df3_year_top3.iloc[2]['app_name']}]

    return rpta3

In [130]:
UsersRecommend(2014)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Counter-Strike: Global Offensive'},
 {'Puesto 3': "Garry's Mod"}]

#### ENDPOINT 4

def **UsersWorstDeveloper**( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [137]:
def UsersWorstDeveloper( anio : int ):
    df4 = pd.read_csv('UsersWorstDeveloper.csv')
    
    # Filtra por el año indicado
    df4_year = df4[df4['year']==anio]
    
    # Ordena de forma ascendente según la columna 'number_user_id_recom' y developer
    sorted_df = df4_year.sort_values(by=['number_user_id_norecom', 'developer'])
    # Elimina duplicados de 'developer', manteniendo la primera ocurrencia, esto se hace para no repetir de desarrolladora
    unique_developers = sorted_df.drop_duplicates(subset='developer', keep='first')
    # selecciona las tres filas con los valores más bajos en la columna 'number_user_id_recom'
    df4_year_top3_developers = unique_developers.head(3)

    # Armamos la estuctura de respuesta
    rpta4 = [{"Puesto 1": df4_year_top3_developers.iloc[0]['developer']},{"Puesto 2": df4_year_top3_developers.iloc[1]['developer']},{"Puesto 3": df4_year_top3_developers.iloc[2]['developer']}]

    return rpta4 

In [141]:
UsersWorstDeveloper(2012)

[{'Puesto 1': 'Best Way'},
 {'Puesto 2': 'Hardsuit Labs'},
 {'Puesto 3': 'NeoCoreGames'}]

#### ENDPOINT 5

def **sentiment_analysis**( empresa desarrolladora : str ): Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

In [184]:
def sentiment_analysis(empresa_desarrolladora: str):
    df5 = pd.read_csv('sentiment_analysis.csv')
    #Filtra por empresa desarrolladora
    df_5_1 = df5[df5['developer']==empresa_desarrolladora]
    # Guarda como diccionario la fila seleccionada
    #rpta5= df_5_1.set_index('developer').to_dict(orient='index')
    #Se da el formato para la respuesta
    rpta5= {row['developer']: [f"Negative = {row['Negative']}", f"Neutral = {row['Neutral']}", f"Positive = {row['Positive']}"] for index, row in df_5_1.iterrows()}

    return rpta5

In [181]:
sentiment_analysis("Valve")

{'Valve': ['Negative = 940', 'Neutral = 780', 'Positive = 4752']}

In [183]:
sentiment_analysis("Valve")

{'Valve': {'Negative': 940, 'Neutral': 780, 'Positive': 4752}}